In [ ]:
import cv2
from typing import Any
import subprocess
import os
from cv2.typing import MatLike
from argparse import ArgumentParser
import numpy as np
from mirage.mirage_helpers import *
from mirage.pose_extract_base import MLAbstractInterface
from mirage.rgb_interface import CameraInterface
from mirage.movenet import MovenetInterface
from mirage.skeleton import SkeletonDetection

%matplotlib inline
from matplotlib import pyplot as plt

def showim(im):
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
    plt.show()

def show_images(images, cols = 1, titles = None):
    """Display a list of images in a single figure with matplotlib.
    
    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.
    
    cols (Default = 1): Number of columns in figure (number of rows is 
                        set to np.ceil(n_images/float(cols))).
    
    titles: List of titles corresponding to each image. Must have
            the same length as titles.
    """
    assert((titles is None)or (len(images) == len(titles)))
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, int(np.ceil(n_images/float(cols))), n + 1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        a.set_title(title)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()

In [ ]:
import yaml
def ost_to_np(filename):
    with open(filename) as s:
        try:
            q = yaml.safe_load(s)
            return {
                "image_width": q["image_width"],
                "image_height": q["image_height"],
                "camera_name": q["camera_name"],
                "camera_matrix": np.array(q["camera_matrix"]["data"]).reshape(
                    (q["camera_matrix"]["rows"], q["camera_matrix"]["cols"])
                ),
                "distortion_coefficients": np.array(q["distortion_coefficients"]["data"]).reshape(
                    (q["distortion_coefficients"]["rows"], q["distortion_coefficients"]["cols"])
                ),
            }
        except yaml.YAMLError as exc:
            print(exc)
            return None


def process_and_viz_split(i_cam: CameraInterface, ml_interface: MLAbstractInterface) -> MatLike:
    img = i_cam.get_next_frame()
    img_a, img_b = split_image_stack(img)
    a_crop = determine_crop_region(s_a, img_a.shape[0], img_a.shape[1])
    b_crop = determine_crop_region(s_b, img_b.shape[0], img_b.shape[1])
    img_a_kp = ml_interface.predict(img_a, a_crop)
    img_b_kp = ml_interface.predict(img_b, b_crop)
    s_a.update_predictions(img_a_kp, (1.0 / i_cam.get_frame_rate_per_second()))
    s_b.update_predictions(img_b_kp, (1.0 / i_cam.get_frame_rate_per_second()))
    img_a_viz = skeleton_to_image(img_a, s_a)
    img_b_viz = skeleton_to_image(img_b, s_b)
    return stack_image(img_a_viz, img_b_viz)


def process_and_viz(i_cam: CameraInterface, ml_interface: MLAbstractInterface) -> MatLike:
    img = i_cam.get_next_frame()
    img_kp = ml_interface.predict(img)
    s_a.update_predictions(img_kp, (1.0 / i_cam.get_frame_rate_per_second()))
    img_viz = skeleton_to_image(img, s_a)
    return img_viz


In [ ]:
input_file = "/home/kadyncbr/Viola02.mp4"
frame_number_start = 0
frame_number_end = 0
frames = []
splitimage = True

In [ ]:
from mirage.skeleton import SkeletonDetection3D
# main
ml_interface = MovenetInterface()
s_a = SkeletonDetection()
s_b = SkeletonDetection()
Skeleton3D = SkeletonDetection3D()
h_skeletons = []
i_cam = CameraInterface(input_file)
i_cam.set_frame(frame_number_start)
frame_number_end = frame_number_end if frame_number_end != 0 else i_cam.get_total_frames()
##### reset

In [ ]:
def display_log_info(im: MatLike, skele: SkeletonDetection):
    img = im.copy()
    num_joints = len(skele.joints)
    print(num_joints)
    starting_y = 150
    starting_x = im.shape[1]-350
    height_per_text = 30
    img = overlay_rect(img, starting_x - 30, 20, im.shape[1] - starting_x+20, im.shape[0] - 40)
    for j in skele.joints.values():
        out = f"{j.name}: {j.confidence*100:.0f}%"
        img = cv2.putText(img, f"{out:<20}", (starting_x, starting_y), cv2.FONT_HERSHEY_PLAIN, 2, (255,0,255), 2)
        starting_y += height_per_text
    return img

def overlay_rect(img: MatLike, x: int, y: int, w: int, h: int):
    # First we crop the sub-rect from the image
    sub_img = img[y:y+h, x:x+w]
    white_rect = np.ones(sub_img.shape, dtype=np.uint8) * 0
    res = cv2.addWeighted(sub_img, 0.25, white_rect, 0.75, 1.0)
    # Putting the image back to its position
    img[y:y+h, x:x+w] = res
    return img

def merge_skeleton(image: MatLike, skeleton_front: SkeletonDetection, skeleton_side: SkeletonDetection):
    homogenous_skeleton = {}
    for k in skeleton_front.joints.keys():
        x, _, y, _ = skeleton_front.joints[k].estimate
        x, y = k_coord(image, (y, x, 1))
        z, _, y2, _ = skeleton_side.joints[k].estimate
        z, y2 = k_coord(image, (y2, z, 1))
        # print(f"Naive Bone matching: {skeleton_front.joints[k].name} [{x}, {y}, {z}]")
        # An issue arrises here when the subject turns or rotates, the 'vertical'(y in the 2d image case) value is no longer grabbing
        # the most reliable frame of reference, Should I continue this, This would be something to change that's largely solved by 
        # reliable depth data or triangulation.
        homogenous_skeleton[k] = [x, y, z]
    return homogenous_skeleton

# Import dependencies
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()

def plot_homogeneous_skeleton(hskeletonDict):
    trace = go.Scatter3d(
        x=[n[0] for n in list(hskeletonDict.values())] + [0, 1280],
        y=[n[2] for n in list(hskeletonDict.values())] + [0, 1280],  
        z=[1280-n[1]-(720) for n in list(hskeletonDict.values())] + [-20, 720], # <-- Make min Z based on estimates?
        mode='markers',
        marker={
            'size': 3,
            'opacity': 0.8,
        }
    )
    linetraces = []
    for p1, p2 in KeypointEdges.keys():
        linetrace = go.Scatter3d(
            x=[n[1][0] for n in list(hskeletonDict.items()) if n[0] == p1 or n[0] == p2],
            y=[n[1][2] for n in list(hskeletonDict.items()) if n[0] == p1 or n[0] == p2],  
            z=[1280-n[1][1]-(720) for n in list(hskeletonDict.items()) if n[0] == p1 or n[0] == p2], 
            mode='lines',
            marker={
                'size': 3,
                'opacity': 0.8,
            }
        )
        linetraces.append(linetrace)
    layout = go.Layout(margin={'l': 0, 'r': 0, 'b': 0, 't': 0})
    data = [trace] + linetraces
    return data
    plot_figure = go.Figure(data=data, layout=layout)
    # Render the plot.
    # plotly.offline.iplot(plot_figure)  

def rotate_z(x, y, z, theta):
    w = x+1j*y
    return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

def plot_homogeneous_skeleton_animation(hskeletonDict_list):
    frames = []
    for hskeletonDict in hskeletonDict_list:
        trace = go.Scatter3d(
            x=[n[0] for n in list(hskeletonDict.values())] + [0, 1280],
            y=[n[2] for n in list(hskeletonDict.values())] + [0, 1280],
            z=[1280-n[1]-(720) for n in list(hskeletonDict.values())] + [-20, 720], # <-- Make min Z based on estimates?
            mode='markers',
            marker={
                'size': 3,
                'opacity': 0.8,
            }
        )
        linetraces = []
        for p1, p2 in KeypointEdges.keys():
            linetrace = go.Scatter3d(
                x=[n[1][0] for n in list(hskeletonDict.items()) if n[0] == p1 or n[0] == p2],
                y=[n[1][2] for n in list(hskeletonDict.items()) if n[0] == p1 or n[0] == p2],
                z=[1280-n[1][1]-(720) for n in list(hskeletonDict.items()) if n[0] == p1 or n[0] == p2],
                mode='lines',
                marker={
                    'size': 3,
                    'opacity': 0.8,
                }
            )
            linetraces.append(linetrace)
        
        frame_data = [trace] + linetraces
        frames.append(go.Frame(data=frame_data))
    layout = go.Layout(
        margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
        height=720,
        updatemenus=[{
            "buttons": [
                {"args": [None, {"frame": {"duration": 30, "redraw": True}, "fromcurrent": True}],
                 "label": "Play",
                 "method": "animate"},
                {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                 "label": "Pause",
                 "method": "animate"}
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }]
    )
    initial_data = plot_homogeneous_skeleton(hskeletonDict_list[0])
    plot_figure = go.Figure(data=initial_data, layout=layout, frames=frames)
    plot_figure.update_layout(
        scene=dict(
            xaxis=dict(range=[0, 1280]),  # adjust x-axis range if necessary
            yaxis=dict(range=[0, 1280]),  # adjust y-axis range if necessary
            zaxis=dict(range=[-50, 720]),  # adjust z-axis range if necessary
            aspectmode='manual',
            aspectratio=dict(x=2,y=2,z=1),
            camera=dict(
                up=dict(x=0, y=0, z=1),
                eye=dict(x=-1.5, y=-1.5, z=0.5)
            )
        ),
        uirevision=True,
    )
    plotly.offline.iplot(plot_figure)

In [ ]:
def get_homogeneous_skeleton3D_plot_data(skeleton_3d_model: SkeletonDetection3D, frame_number: int, zmin: int=-40, zmax: int=120):
    def zoff(z:int):
        return 1280-z-720
    X_IND = 0
    Y_IND = 4
    Z_IND = 2
    joint_saved_states = [j.get_estimate_at(frame_number) for j in skeleton_3d_model.joints.values()]
    # Plot joint points 3D
    trace = go.Scatter3d(
        x=[j[X_IND] for j in joint_saved_states] + [0, 1280],
        y=[j[Y_IND] for j in joint_saved_states] + [0, 1280],  
        z=[zoff(j[Z_IND]) for j in joint_saved_states] + [zmin, zmax], # <-- Make min Z based on estimates?
        mode='markers',
        marker={
            'size': 3,
            'opacity': 0.8,
        }
    )
    # Plot joint connections 3D
    linetraces = []
    for p1, p2 in KeypointEdges.keys():
        p1Joint = skeleton_3d_model.joints[p1]
        p2Joint = skeleton_3d_model.joints[p2]
        linetrace = go.Scatter3d(
            x=[p1Joint.get_estimate_at(frame_number)[X_IND]] + [p2Joint.get_estimate_at(frame_number)[X_IND]],
            y=[p1Joint.get_estimate_at(frame_number)[Y_IND]] + [p2Joint.get_estimate_at(frame_number)[Y_IND]],
            z=[zoff(p1Joint.get_estimate_at(frame_number)[Z_IND])] + [zoff(p2Joint.get_estimate_at(frame_number)[Z_IND])], 
            mode='lines',
            marker={
                'size': 3,
                'opacity': 0.8,
            }
        )
        linetraces.append(linetrace)
    data = [trace] + linetraces
    return data
    
def plot_homogeneous_skeleton3D_animation(skeleton_3d_model: SkeletonDetection3D, zmin: int=-40, zmax: int=120):
    frames = []
    print("Grabbing frame")
    for frame_num in range(len(skeleton_3d_model.joints[0].saved_states)):
        frame_data = get_homogeneous_skeleton3D_plot_data(skeleton_3d_model, frame_num, zmin, zmax)
        frames.append(go.Frame(data=frame_data))
    print("all frames grabbed.")
    layout = go.Layout(
        margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
        height=720,
        updatemenus=[{
            "buttons": [
                {"args": [None, {"frame": {"duration": 30, "redraw": True}, "fromcurrent": True}],
                 "label": "Play",
                 "method": "animate"},
                {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                 "label": "Pause",
                 "method": "animate"}
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }]
    )
    initial_data = get_homogeneous_skeleton3D_plot_data(skeleton_3d_model, 0, zmin, zmax)
    plot_figure = go.Figure(data=initial_data, layout=layout, frames=frames)
    plot_figure.update_layout(
        scene=dict(
            xaxis=dict(range=[0, 1280]),  # adjust x-axis range if necessary
            yaxis=dict(range=[0, 1280]),  # adjust y-axis range if necessary
            zaxis=dict(range=[-50, 720]),  # adjust z-axis range if necessary
            aspectmode='manual',
            aspectratio=dict(x=2,y=2,z=1),
            camera=dict(
                up=dict(x=0, y=0, z=1),
                eye=dict(x=-1.5, y=-1.5, z=0.5)
            )
        ),
        uirevision=True,
    )
    plotly.offline.iplot(plot_figure)

In [ ]:
for i in range(frame_number_end):
    imgs = process_and_viz_split(i_cam, ml_interface)
    im_a, im_b = split_image_stack(imgs)
    # show_images([im_a, im_b])
    hskeleton = merge_skeleton(im_a, s_a, s_b)
    h_skeletons.append(hskeleton)
    Skeleton3D.update_predictions(hskeleton, (1.0 / i_cam.get_frame_rate_per_second()))

In [ ]:
import pickle # lazy serialization
pickl = pickle.dump(Skeleton3D, open(f"{input_file.split('/')[-1].split('.')[0]}.sk3d", "wb"))
lazyreconstruct = pickle.load(open(f"{input_file.split('/')[-1].split('.')[0]}.sk3d", "rb"))

plot_homogeneous_skeleton3D_animation(lazyreconstruct)
# plot_homogeneous_skeleton3D_animation(Skeleton3D)